# Dimensionality reduction with t-SNE

t-Distributed Stochastic Neighbor Embedding (t-SNE) is a technique for dimensionality reduction that is particularly well suited for the [visualization of high-dimensional datasets](https://lvdmaaten.github.io/tsne/). It was created by [van der Maaten and Hinton](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf) in 2008. 

In this notebooks, we are going to show how t-SNE works using features extracted from the kaggle dataset [dogs vs cats](https://www.kaggle.com/c/dogs-vs-cats/). For extracting the features, we will use ResNet50 as featurizer. 

We present and compare two implementations of t-SNE: [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), implemented in python and [CUDA](https://github.com/georgedimitriadis/t_sne_bhcuda), implemented by George Dimitriadis in CUDA/C++ with python wrappers.


In [1]:
import os
import sys
import numpy as np
import sklearn
import keras as K
from keras.applications.nasnet import NASNetLarge
from keras.applications.resnet50 import ResNet50
import tensorflow as tf
import kaggle
from numba import cuda
import t_sne_bhcuda.t_sne_bhcuda.bhtsne_cuda as tsne_bhcuda
from utils import (plot_tsne, get_gpu_name, get_cuda_version, get_cudnn_version,
                   find_files_with_pattern, featurize_images, clear_memory_all_gpus)

print("System version: {}".format(sys.version))
print("Sklearn version: {}".format(sklearn.__version__))
print("Numpy version: {}".format(np.__version__))
print("Kaggle version: {}".format(kaggle.KaggleApi.__version__))
print("Keras version: {}".format(K.__version__))
print("Keras backend: {}".format(K.backend.backend()))
print("Keras image data format: {}".format(K.backend.image_data_format()))
print("Tensorflow version: {}".format(tf.__version__))
print("GPU: {}".format(get_gpu_name()))
print("CUDA version: {}".format(get_cuda_version()))
print("CuDNN version: {}".format(get_cudnn_version()))


# Autoreload changes in imported files
%load_ext autoreload
%autoreload 2

# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Using TensorFlow backend.


System version: 3.5.5 |Anaconda custom (64-bit)| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]
Sklearn version: 0.19.1
Numpy version: 1.14.5
Kaggle version: 1.5.0
Keras version: 2.2.2
Keras backend: tensorflow
Keras image data format: channels_last
Tensorflow version: 1.10.1
GPU: ['Tesla K80']
CUDA version: CUDA Version 9.2.148
CuDNN version: 7.2.1


### Dataset

[Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats/data) dataset, which contains 2 classes and 25000 images.

Make sure you follow the [instructions](https://github.com/Kaggle/kaggle-api#api-credentials) to get the Kaggle credentials. 


In [2]:
!/anaconda/envs/py35/bin/kaggle competitions download -c dogs-vs-cats --force

  0%|                                               | 0.00/86.8k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 86.8k/86.8k [00:00<00:00, 2.14MB/s]
 97%|███████████████████████████████████████▊ | 263M/271M [00:01<00:00, 158MB/s]
100%|█████████████████████████████████████████| 271M/271M [00:02<00:00, 115MB/s]
 99%|███████████████████████████████████████▌| 537M/543M [00:06<00:00, 81.6MB/s]
100%|████████████████████████████████████████| 543M/543M [00:11<00:00, 50.8MB/s]


In [3]:
!unzip -q train.zip

In [4]:
files_dog = find_files_with_pattern("train", "dog*")
len(files_dog)
print(files_dog[:10])

files_cat = find_files_with_pattern("train", "cat*")
len(files_cat)
print(files_cat[:10])

12500

['train/dog.7647.jpg', 'train/dog.4124.jpg', 'train/dog.11342.jpg', 'train/dog.7021.jpg', 'train/dog.7305.jpg', 'train/dog.1757.jpg', 'train/dog.1804.jpg', 'train/dog.8312.jpg', 'train/dog.2823.jpg', 'train/dog.1423.jpg']


12500

['train/cat.6586.jpg', 'train/cat.11278.jpg', 'train/cat.8459.jpg', 'train/cat.1983.jpg', 'train/cat.5735.jpg', 'train/cat.3765.jpg', 'train/cat.10067.jpg', 'train/cat.6049.jpg', 'train/cat.783.jpg', 'train/cat.8995.jpg']


In [5]:
sample = 5000
files_dog = files_dog[:sample]
files_cat = files_cat[:sample]
file_names = files_dog + files_cat
len(file_names)

10000

In [6]:
labels_dog = [0]*len(files_dog)
labels_cat = [1]*len(files_cat)
labels = labels_dog + labels_cat
len(labels)

10000

### Image featurization

In [7]:
#https://keras.io/applications/#resnet50
model = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False, pooling='avg')

In [8]:
features = featurize_images(file_names, model)
features.shape

157it [01:51,  1.56it/s]


(10000, 2048)

In [9]:
# clear gpu memory
del model
clear_memory_all_gpus()

### Dimensionality reduction with TSNE

In [10]:
perplexity = 10.0
theta = 0.5
learning_rate = 200.0
iterations = 2000
gpu_mem = 0.95
files_dir='tsne_results'

In [ ]:
%%time
t_sne_result_sklearn = tsne_bhcuda.t_sne(samples=features, use_scikit=True, files_dir=files_dir,
                        no_dims=2, perplexity=perplexity, eta=learning_rate, theta=theta,
                        iterations=iterations, gpu_mem=gpu_mem, randseed=-1, verbose=2)

[t-SNE] Computing 31 nearest neighbors...
[t-SNE] Indexed 10000 samples in 1.394s...
[t-SNE] Computed neighbors for 10000 samples in 385.605s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 5.098726
[t-SNE] Computed conditional probabilities in 0.778s
[t-SNE] Iteration 50: error = 28.0392838, gradient norm = 0.0114708 (50 iterations in 10.638s)
[t-SNE]

In [ ]:
plot_tsne(t_sne_result_sklearn, labels)

In [ ]:
%%time
t_sne_result_gpu = tsne_bhcuda.t_sne(samples=features, use_scikit=False, files_dir=files_dir,
                        no_dims=2, perplexity=perplexity, eta=learning_rate, theta=theta,
                        iterations=iterations, gpu_mem=gpu_mem, randseed=-1, verbose=2)

In [ ]:
plot_tsne(t_sne_result_gpu, labels)

In [ ]:
!rm -rf train